# MLP w TensorFlow



In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

C:\Users\admin\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Zad.




In [13]:
n_inputs = 28*28 # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [14]:
from functools import partial

batch_norm_momentum = 0.9

tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

training = tf.placeholder_with_default(False, shape=(), name='training')


with tf.name_scope("dnn"):
    
    he_init = tf.contrib.layers.variance_scaling_initializer()
    
    my_batch_norm_layer = partial(
            tf.layers.batch_normalization,
            training=training,
            momentum=batch_norm_momentum)

    my_dense_layer = partial(
            tf.layers.dense,
            kernel_initializer=he_init)
    
    hidden1 = my_dense_layer(X, n_hidden1, name="hidden1")
    bn1 = tf.nn.elu(my_batch_norm_layer(hidden1))
    hidden2 = my_dense_layer(bn1, n_hidden2, name="hidden2")
    bn2 = tf.nn.elu(my_batch_norm_layer(hidden2))
    logits_before_bn = my_dense_layer(bn2, n_outputs, name="outputs")
    logits = my_batch_norm_layer(logits_before_bn)     

In [15]:
with tf.name_scope("loss"):
    xentropy = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss") 

In [16]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)  

In [17]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, tf.argmax(y, 1), 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))   

In [18]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [19]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [20]:
n_epochs = 100
batch_size = 64

In [21]:
# extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

# with tf.Session() as sess:
#     sess.run(init)
#     for epoch in range(n_epochs):
#         for batch_index in range(mnist.train.num_examples // batch_size):
#             print("\r{}%".format(100 * batch_index //  (mnist.train.num_examples // batch_size) ), end="")
#             X_batch, y_batch = mnist.train.next_batch(batch_size)
#             sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
#         acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
#         acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
#         print(" ", epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)    
        
#     #save_path = saver.save(sess, "./MNIST_MLP/my_model_final_one_hot.ckpt")   

In [35]:
with tf.Session() as sess:
    saver.restore(sess, "./MNIST_MLP/my_model_final_one_hot.ckpt") 
    for epoch in range(5):
        for batch_index in range(mnist.train.num_examples // batch_size):
            print("\r{}%".format(100 * batch_index //  (mnist.train.num_examples // batch_size) ), end="")
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
        print(" ", epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

INFO:tensorflow:Restoring parameters from ./MNIST_MLP/my_model_final_one_hot.ckpt
99%  0 Train accuracy: 1.0 Test accuracy: 0.976
99%  1 Train accuracy: 1.0 Test accuracy: 0.9758
99%  2 Train accuracy: 1.0 Test accuracy: 0.9756
99%  3 Train accuracy: 1.0 Test accuracy: 0.9757
99%  4 Train accuracy: 1.0 Test accuracy: 0.9755


# Możesz po prostu ponownie użyć części, których potrzebujesz i opuść resztę:

In [38]:
n_inputs = 28*28 # MNIST
n_hidden1 = 300 # reused
n_hidden2 = 10 # new!
n_outputs = 10  # new!

from functools import partial

batch_norm_momentum = 0.9

tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

training = tf.placeholder_with_default(False, shape=(), name='training')


with tf.name_scope("dnn"):
    
    he_init = tf.contrib.layers.variance_scaling_initializer()
    
    my_batch_norm_layer = partial(
            tf.layers.batch_normalization,
            training=training,
            momentum=batch_norm_momentum)

    my_dense_layer = partial(
            tf.layers.dense,
            kernel_initializer=he_init)
    
    hidden1 = my_dense_layer(X, n_hidden1, name="hidden1")            # reused
    bn1 = tf.nn.elu(my_batch_norm_layer(hidden1))                     # reused
    hidden2 = my_dense_layer(bn1, n_hidden2, name="hidden2")          # new!
    bn2 = tf.nn.elu(my_batch_norm_layer(hidden2))                     # new!
    logits_before_bn = my_dense_layer(bn2, n_outputs, name="outputs") # new!
    logits = my_batch_norm_layer(logits_before_bn)                    # new!
    
with tf.name_scope("loss"):
    xentropy = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")     
    
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)      
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, tf.argmax(y, 1), 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))      

In [49]:
reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                               scope="hidden[1]") # regular expression
restore_saver = tf.train.Saver(reuse_vars) # to restore layers 1-3

extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    restore_saver.restore(sess, "./MNIST_MLP/my_model_final_one_hot.ckpt") 
    for epoch in range(5):
        for batch_index in range(mnist.train.num_examples // batch_size):
            print("\r{}%".format(100 * batch_index //  (mnist.train.num_examples // batch_size) ), end="")
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
        print(" ", epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

INFO:tensorflow:Restoring parameters from ./MNIST_MLP/my_model_final_one_hot.ckpt
99%  0 Train accuracy: 0.9375 Test accuracy: 0.9026
99%  1 Train accuracy: 0.921875 Test accuracy: 0.9227
99%  2 Train accuracy: 0.953125 Test accuracy: 0.9338
99%  3 Train accuracy: 0.984375 Test accuracy: 0.9415
99%  4 Train accuracy: 0.953125 Test accuracy: 0.9477
